# Proyecto Integrador

## Introduccion al tipo de algoritmo.

### Algoritmo de Centralidad
#### Cercania Centralidad

Estos tipos de algoritmo hacen referencia a aquellos nodos con una posicion mas central, es decir que tienen una acceso mas facil dentro del arbol de nodos o grafo. Este tipo de algoritmo sirve para tener un acceso mas rapito y facil en cuestion de flujo dentro del arbol de nodos. 
Cercanía (closeness) es el promedio de las distancias del vértice a todos los demás nodos.
La medida de centralidad por cercanía debe tener mayor valor para nodos más centrales y por esto se considerarán los inversos del promedio de las distancias para su definición, como se observa en la ecuación

CC(x) =N-1n / ∑ d(y,x)

Esta ecuacion representa el calculo de la cercania centralidad donde N es el numero de nodos  y se divide para la Sumatoria de la distancia entre los demas nodos

### Descricion del algoritmo. 
Este algoritmo es utilizado para saber que tipo nodo se encuentra mas central al grafo con una menor distancia, se puede decir que el mismo esta en una posicion favorable para poder aaceder a los demas nodos u obtener informacion. Se dice que es el nodo que por el que puede fluir mayor trafico o por donde se pasaran la mayoria de trayectorias.


## Proceso (Paso a paso del uso del algoritmo)

Para el calculo del algoritmo tenemos 4 nodos:
A-B-C-D
El nodo A esta conectado al nodo C y B
El nodo B esta conectado al nodo C Y A
El nodo C esta conextado al nodo A,B Y D
El nodo D esta conectado al nodo C

para el calculo de la centralidade del algoritmo aplicamos la ecuacion donde para:

El nodo A:

4-1/4 = 0.75

El nodo B:

4-1/4 = 0.75

El nodo C:

4-1/3= 1.0

El nodo D:

3-1/3 = 0.6

## Ejemplos sencillos. 

Para realizar la ejemplificacion utilizaremos neo4j en donde procederemos a crear 5 nodos conectados entre si para lo cual procedemos 
a conectarnos a nuestra basee de neo4j e ingresamos los nodos

CREATE (a:Node{id:"A"}),
       (b:Node{id:"B"}),
       (c:Node{id:"C"}),
       (d:Node{id:"D"}),
       (e:Node{id:"E"}),
       (a)-[:LINK]->(b),
       (b)-[:LINK]->(a),
       (b)-[:LINK]->(c),
       (c)-[:LINK]->(b),
       (c)-[:LINK]->(d),
       (d)-[:LINK]->(c),
       (d)-[:LINK]->(e),
       (e)-[:LINK]->(d);
 
 #### Resultado de Neo4j
 
 <img src="centralidad.png">
 
Luego procedemos a llamar a nuestro metodo para consultar la centralidad de cada nodo

CALL gds.alpha.closeness.stream({
  nodeProjection: 'Node',
  relationshipProjection: 'LINK'
})
YIELD nodeId, centrality
RETURN gds.util.asNode(nodeId).name AS user, centrality
ORDER BY centrality DESC

In [7]:
from neo4j import GraphDatabase

uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "malki"),encrypted=False)
    
personas = []
centralidad = []
def get_mo(tx):
    result = tx.run("CALL gds.alpha.closeness.stream({nodeProjection: 'Node', "
                    "relationshipProjection: 'LINK' })"
                    "YIELD nodeId, centrality "
                    "RETURN gds.util.asNode(nodeId).id AS user, centrality "
                    "ORDER BY centrality DESC")
    for record in result:
        personas.append(record["user"])
        centralidad.append(record["centrality"])

print("Cercania Centralidad")
print("-------------------------------------------------------------")
with driver.session() as session:
    ciudades = session.read_transaction(get_mo)
    print("Solución")
    print("Nodo -> Centralidad")
    for i in range(len(personas)):
        print(personas[i], " -> ", centralidad[i])                       
driver.close()

Cercania Centralidad
-------------------------------------------------------------
Solución
Nodo -> Centralidad
C  ->  0.6666666666666666
B  ->  0.5714285714285714
D  ->  0.5714285714285714
A  ->  0.4
E  ->  0.4


## Ejemplificacion usando datos reales.

Para el ejemplo del uso de datos reales procedemos a tomar informacion de la ciudad de quito de los parques, iglesias, hospitales, bombres, policia, escuelas, centros medicos, lugares turisticos, centros de estimulacion temprana y centros educativos para el desarrollo de ninios. 

Una vez obtenida la informacion procedemos a realizar la creacion de los nodos con cada uno de los datos recolectados y con sus respectivas relaciones entre ellos.

Dentro de nuestra base de neo4j se encuentran creados los nodos de cada lugar requerido de la ciudad de quito. Una vez obtenido procedemos a realizar la busqueda del lugar con mas centralidad dentro de nuestro grafo.

Para este ejemplo utilizaremos el grafo de los parques de quito, el algoritmo nos devolvera el parque que mayor centralidad posee, es decir aquel parque mejor conectado y al que se tendria un acceso mas fluido de acuerdo a las relaciones que posee el arbol construido.

## Resultados y analisis.  

In [6]:
from neo4j import GraphDatabase

uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "malki"),encrypted=False)
    
parques = []
centralidad = []
def get_mo(tx):
    result = tx.run("CALL gds.alpha.closeness.stream({nodeProjection: 'Parque', "
                    "relationshipProjection: 'LINK' })"
                    "YIELD nodeId, centrality "
                    "RETURN gds.util.asNode(nodeId).name AS user, centrality "
                    "ORDER BY centrality DESC")
    for record in result:
        parques.append(record["user"])
        centralidad.append(record["centrality"])

print("Cercania Centralidad")
print("-------------------------------------------------------------")
with driver.session() as session:
    ciudades = session.read_transaction(get_mo)
    print("Solución")
    print("Nodo -> Centralidad")
    for i in range(len(parques)):
        print(parques[i], " -> ", centralidad[i])                       
driver.close()

Cercania Centralidad
-------------------------------------------------------------
Solución
Nodo -> Centralidad
Parque La Alameda  ->  0.3114754098360656
Parque Lineal Machangara  ->  0.2992125984251969
Parque Cumanda  ->  0.2923076923076923
Parque El Ejido  ->  0.2857142857142857
Parque Guapulo  ->  0.2753623188405797
Parque Lineal Quito Sur  ->  0.2695035460992908
Parque Itichimbia  ->  0.26573426573426573
Parque Solanda  ->  0.2620689655172414
Parque Jerusalen Alangasi  ->  0.2602739726027397
Parque La Carolina  ->  0.25675675675675674
Terminal Terrestre Quitumbe  ->  0.24836601307189543
Parque Las Cuadras  ->  0.24675324675324675
Parque EL Turismo  ->  0.2375
Parque Arqueológico Rumipamba  ->  0.2375
Parque Lineal Selva Alegre  ->  0.2345679012345679
Parque Metropolitano La Armeria  ->  0.2275449101796407
Parque Metropolitano Chilibulo  ->  0.2275449101796407
Parque de la Jipijapa  ->  0.22485207100591717
Parque Calicanto  ->  0.22093023255813954
Parque Central Tumbaaco  ->  0.2196

## Conclusiones.
El algoritmo de cercania centralidad puede ir mejorando de acuerdo a las diferentes relaciones que pueda exisitir dentro del grafo, es recomendable el uso de este algoritmo dentro de arboles con relaciones entre nodos, ya que el uso en grafos sin relaciones podria resultar en una busqueda infinita. Los lugares mejor conectados, dependera de la construccion del grafo y el problema que se quiera solucionar. En este caso vemos como el acceso a un lugar el cual se encuentra en un ubicacion central con mayor conexiones entre los diferentes nodos.

### Algoritmo de Busqueda de Ruta
#### Ruta mas corta

Los algoritmos de Busqueda de ruta son aquellos que implementan un busque da un camino, entre un nodo inicial y un nodo final o destino. Hat varios algoritmos de busqueda de ruta entre ellos tenemos al algoritmo de busqueda de ruta mas corta, este algoritmo implementa 
una busqueda para encontrar un camino con el menor coste en distancia entre su nodo origen y final.

El algoritmo de busqueda de la ruta mas corta tiene una gran historia ya que en el sigolo XIX se desarrollaron algoritmos para encontrar un ruta eficiente para aplicarlos en diferentes casos. En 1956 el cientifico Dijkstra se encontraba en la necesidad de realizar la demostracion de las nuevas computadoras ARMAC. El decidio establacerse una problema para realizar la demostracion de las computadoras hacia las personas comunes, que no tenian un grado de conociminiento sobre la Informatica. Para ello se eestablecio el algoritmo de Dijkstra en cual pudo implementar en un mapa de transportes de 64 ciudades.

### Descricion del algoritmo. 

El algoritmo de la busqueda de la ruta mas corta implementa una busquedea de acuerdo a la distancia exisitente entre los nodos origen u un nodo n, mientras mas avanza el algoritmo se va obtienendo los siguiente nodos en funcion del costo producido para llegar al nodo siguiente hasta encontrar una solucion.

Este algoritmo realiza una busqueda de acuerdo a la distancia establecida entre los nodos, esta distancia requiere que sea no negativa, para poder encontrar una solucion mas acertada a la ruta del nodo objetivo. 

Este tipo de algoritmo a tenido gran aceptacion dentro del apartado informatico ya que a dado grandes soluciones y se ah conseguido grandes trabajos como la implementacion de la recomendacion de rutas que brinda Google Maps. 


## Proceso (Paso a paso del uso del algoritmo)
Se tiene un arbol de nodos:
 <img src="rutacorta.png">
 
 para llegar al nodo F desde el nodo A procedemos a crear una lista de Nodos visitado y una lista de nodo por visitar
 
 Visitado={A(0)}
 ListaporVisitar={B(50),D(100)}

 Visitado={A(0),B(50)}
 ListaporVisitar={D(100),C(100)}
 
 Visitado={A(0),B(50),D(100)}
 ListaporVisitar={C(100),E(200)}
 
 Visitado={A(0),B(50),D(100),C(100)}
 ListaporVisitar={C(100),E(200),F(200)}
 
 Ruta = A->B->C->F,
 Solucion Nodo F se encuentra con un costo de 200
 
 ## Ejemplos sencillos. 
 
Para un ejemplo procedemos a crear nuestros nodos en la base de datos neo4j con sus respectivas relaciones y costos de las mismas.

CREATE (a:Loc {name: 'A'}),
       (b:Loc {name: 'B'}),
       (c:Loc {name: 'C'}),
       (d:Loc {name: 'D'}),
       (e:Loc {name: 'E'}),
       (f:Loc {name: 'F'}),
       (a)-[:ROAD {cost: 50}]->(b),
       (a)-[:ROAD {cost: 50}]->(c),
       (a)-[:ROAD {cost: 100}]->(d),
       (b)-[:ROAD {cost: 40}]->(d),
       (c)-[:ROAD {cost: 40}]->(d),
       (c)-[:ROAD {cost: 80}]->(e),
       (d)-[:ROAD {cost: 30}]->(e),
       (d)-[:ROAD {cost: 80}]->(f),
       (e)-[:ROAD {cost: 40}]->(f);

<img src="costo.png">

Luego procedemos a consulta nuestro metodo para encontrar la ruta mas corta entre los nodos.

MATCH (start:Loc {name: 'A'}), (end:Loc {name: 'F'})
CALL gds.alpha.shortestPath.write({
  nodeProjection: 'Loc',
  relationshipProjection: {
    ROAD: {
      type: 'ROAD',
      properties: 'cost',
      orientation: 'UNDIRECTED'
    }
  },
  startNode: start,
  endNode: end,
  weightProperty: 'cost',
  writeProperty: 'sssp'
})
YIELD nodeCount, totalCost
RETURN nodeCount,totalCost


In [14]:
from neo4j import GraphDatabase

uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "malki"),encrypted=False)
    
nodos = []
costo = []
def get_mo(tx):
    result = tx.run("MATCH (start:Loc {name: 'A'}), (end:Loc {name: 'F'}) "
                    "CALL gds.alpha.shortestPath.stream({"
                    "nodeProjection: 'Loc', "
                    "relationshipProjection: { "
                    "ROAD: { "
                    "type: 'ROAD', properties: 'cost',orientation: 'UNDIRECTED' }}, "
                    "startNode: start, endNode: end, relationshipWeightProperty: 'cost' }) "
                    "YIELD nodeId, cost "
                    "RETURN gds.util.asNode(nodeId).name AS name, cost")
    for record in result:
        nodos.append(record["name"])
        costo.append(record["cost"])


with driver.session() as session:
    ciudades = session.read_transaction(get_mo)
    print("Solución")
    print("Nodo -> Costo")
    for i in range(len(nodos)):
        print(nodos[i], " -> ", costo[i])                       
driver.close()

Solución
Nodo -> Costo
A  ->  0.0
B  ->  50.0
D  ->  90.0
E  ->  120.0
F  ->  160.0


## Ejemplificacion usando datos reales.

Para realizar la busqueda de una ruta mas corta implementaremos datos de los hospitales de la ciudad de Quito, para ello procedemos  cargar nuestros datos desde un csv y luego procedemos a crear nuestro nodos con sus respectivas relaciones. 

Una ve que tenemos los nodos crados con sus repectivas rutas procedemos a buscar dentro de nuestro grafo un hostipal de interes y el origen de donde queremos partir. 

El algoritmo nos devolvera la ruta haciena el nodo objetivo con el menor costo.

## Resultados y analisis. 

In [3]:
from neo4j import GraphDatabase

uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "malki"),encrypted=False)
    
nodos = []
costo = []
def get_mo(tx):
    result = tx.run("MATCH (start:Hospital {name: 'Terminal Terrestre Quitumbe'}), (end:Hospital {name: 'Hospital Militar'}) "
                    "CALL gds.alpha.shortestPath.stream({"
                    "nodeProjection: 'Hospital', "
                    "relationshipProjection: { "
                    "ROAD: { "
                    "type: 'LINK', properties: 'cost',orientation: 'UNDIRECTED' }}, "
                    "startNode: start, endNode: end, relationshipWeightProperty: 'cost' }) "
                    "YIELD nodeId, cost "
                    "RETURN gds.util.asNode(nodeId).name AS name, cost")
    for record in result:
        nodos.append(record["name"])
        costo.append(record["cost"])

with driver.session() as session:
    ciudades = session.read_transaction(get_mo)
    print("Solución")
    print("Nodo -> Costo")
    for i in range(len(nodos)):
        print(nodos[i], " -> ", costo[i])                       
driver.close()

Cercania Centralidad
-------------------------------------------------------------
Solución
Nodo -> Costo
Terminal Terrestre Quitumbe  ->  0.0
Hospital Del Seguro  ->  9.8
Hopital del Dia Central Quito IESS  ->  13.9
Hospital de Especialidades Eugenio Espejo  ->  15.6
Hospital Militar  ->  16.25


## Conclusiones.
Este algortimo nos permite encontrar la mejor ruta existente entre dos nodos de un grafo o arbol. El algortimo nos permite devolver la ruta seguida hacia el destino empleando un costo minimo, este tipo de busquedas esta mas enfocado en encontrar rutas existentes entre dos puntos en un espacio o plano cartesiano para poder emplear el menor coste o esfuerzo en alcanzar el objetivo.

### Algoritmo de prediccion de Enlaces
#### Vecinos Comunes

Este tipo de algoritmo nos ayuda a determinar la cercania que existe entre 2 nodos de un arbol o grafo. Dentro de ellos existen varios algoritmos uno de ellos es el Algoritmo de vecinos comunes el cual nos permite determinar elementos en comunes entre dos nodos, Esto expresa entre cuanta posibilidad hay que un nodo pueda comunicase con otro de acuerdo a los nodos hijos que tenga. 

### Descricion del algoritmo. 

El algoritmo de vecino comunes implementa una formula de conjuntos:

    CN(x,y) =|N(x) interseccion N(y)|

Este tipo de algoritmo nos permite determinar si dos nodos comparten nodos en comun. Este tipo de algortimo se ha empleado para determinar la probabilidad de que dos nodos que comparten otros nodos en comun puedan encontrarse. Es decir si una persona es amigo de una persona n y otra persona es igual amigo de la persona n, la persona 1 y la persona 2 tienen un grado de probabilidad de conocerse por tener un amigo en comun.

## Proceso (Paso a paso del uso del algoritmo)

Para demostras el algoritmo de vecinos comunes implementaremos la formula en dos conjutos:

A= {1,2,3,4,5}
B= {9,8,4,3,7}

Al aplicar la formula de los vecinos comunes entre A interseccion B
obtenemos que ellos tienen en comun:

A n B = {4,3}

 ## Ejemplos sencillos.
 
 Para realizar un ejemplo de este algortimo procedemos a crar los siguiente nodos en nuestra base de Datos.

CREATE
 (zhen:Person {name: 'Zhen'}),
 (praveena:Person {name: 'Praveena'}),
 (michael:Person {name: 'Michael'}),
 (arya:Person {name: 'Arya'}),
 (karin:Person {name: 'Karin'}),

 (zhen)-[:FRIENDS]->(arya),
 (zhen)-[:FRIENDS]->(praveena),
 (praveena)-[:WORKS_WITH]->(karin),
 (praveena)-[:FRIENDS]->(michael),
 (michael)-[:WORKS_WITH]->(karin),
 (arya)-[:FRIENDS]->(karin)
 
Luego de crear nuestros nodos procedemos a correr la consulta que nos permite determinar si dos nodos tienes vecinos comunes, es decir este algoritmo devolvera 1 si tienen nodos en comun y 0 si no tienen nada en comun 

MATCH (p1:Person {name: 'Michael'})
MATCH (p2:Person {name: 'Karin'})
RETURN gds.alpha.linkprediction.commonNeighbors(p1, p2) AS score



In [13]:
from neo4j import GraphDatabase

uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "malki"),encrypted=False)
    
nodos = []
costo = []
def get_mo(tx):
    result = tx.run("MATCH (p1:Person {name: 'Michael'}) "
                    "MATCH (p2:Person {name: 'Karin'}) "
                    "RETURN gds.alpha.linkprediction.commonNeighbors(p1, p2) AS score")
    for record in result:
        nodos.append(record["score"])
        
with driver.session() as session:
    ciudades = session.read_transaction(get_mo)
    print("Solución")
    print("Vecinos Comunes")
    print(nodos[0])                     
driver.close()

Solución
Vecinos Comunes
1.0


## Ejemplificacion usando datos reales.

Para realizar el ejemplo porcedemos a realizar nuestros nodos para ello utilizaremos los lugares de quito que en este caso seran dos parques diferentes de los cuales obtendremos si existen vecinos comunes entre ellos.

Una vez cargado nueestro nodos procedemos a realizar la consulta a nuestra base de datos neo4j, en la cual tendremos nuestros nodos con sus repectivos datos. 

## Resultados y analisis. 

In [15]:
from neo4j import GraphDatabase

uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "malki"),encrypted=False)
    
nodos = []
costo = []
def get_mo(tx):
    result = tx.run("MATCH (p1:Parque {name: 'Parque La Alameda'}) "
                    "MATCH (p2:Parque {name: 'Parque Jerusalen Alangasi'}) "
                    "RETURN gds.alpha.linkprediction.commonNeighbors(p1, p2) AS score")
    for record in result:
        nodos.append(record["score"])
        
with driver.session() as session:
    ciudades = session.read_transaction(get_mo)
    print("Solución")
    print("Vecinos Comunes")
    print(nodos[0])                     
driver.close()

Solución
Vecinos Comunes
1.0


## Conclusiones.
Este tipo de algoritmo es implementado para la busqueda de 2 nodos desconocidos en donde puede existir vecinos comunes, mientras mas cerca el valor del algoritmo este de 1 mas cercanos ceran, si el algoritmo devuelve el valor de 0, esto quiere decir que no se puede dar que los dos nodos tenga algo en comun. 

### Algoritmo de vecinos mas cercanos aproximados (ANN)

El Algoritmo Approximate Nearest Neighbors construye un gráfico de vecinos más cercanos para un conjunto de objetos basado en un algoritmo de similitud proporcionado. La similitud de los elementos se calcula en función de la similitud de Jaccard,la similitud de coseno, ladistancia euclidianao la similitud de Pearson.

Podemos usar el algoritmo Approximate Nearest Nearest Neighbors para averiguar los elementos k más similares entre sí. El gráfico de vecinos más cercanos se puede utilizar como parte de las consultas de recomendación.

### Descripcion del Algoritmo

Como se ha mencionado anteriormente este algoritmo calcula la similitud en funcion de la similitud de jeccard, la similitud de coseno, la distancia euclidiana y la similitud de Pearson. En este caso nos centraremos en la similitud de jaccard en donde se utiliza la siguiente formula: 
                    J(A,B) = ∣A ∩ B∣ / ∣A∣ + ∣B∣ - ∣A ∩ B|

## Proceso (Paso a paso del uso del algoritmo)

Para hacer una demostracion de como trabaja el algoritmo con la similitud de jaccard  
utilizaremos los siguientes dos conjuntos de datos. 

A= {1,2,3}
B= {1,2,4,5} 

En donde implementaremos la formula la cual nos dice que es igual a la interseccion del conjunto A con el conjunto B dividido con la union del conjunto A con el conjunto B. Lo que quedaria de la siguiente forma. 

J(A,B) = ∣A ∩ B∣ / ∣A∣ + ∣B∣ - ∣A ∩ B|
J(A,B) = 2 / 3 + 4 - 2
       = 2 / 5
       = 0.4

## Ejemplo Sencillo

Para realizar un ejemplo sencillo procedemos a crear los siguientes nodos con sus relaciones en la base de datos 
CREATE
  (french:Cuisine {name:'French'}),
  (italian:Cuisine {name:'Italian'}),
  (indian:Cuisine {name:'Indian'}),
  (lebanese:Cuisine {name:'Lebanese'}),
  (portuguese:Cuisine {name:'Portuguese'}),

  (zhen:Person {name: 'Zhen'}),
  (praveena:Person {name: 'Praveena'}),
  (michael:Person {name: 'Michael'}),
  (arya:Person {name: 'Arya'}),
  (karin:Person {name: 'Karin'}),

  (praveena)-[:LIKES]->(indian),
  (praveena)-[:LIKES]->(portuguese),

  (zhen)-[:LIKES]->(french),
  (zhen)-[:LIKES]->(indian),

  (michael)-[:LIKES]->(french),
  (michael)-[:LIKES]->(italian),
  (michael)-[:LIKES]->(indian),

  (arya)-[:LIKES]->(lebanese),
  (arya)-[:LIKES]->(italian),
  (arya)-[:LIKES]->(portuguese),

  (karin)-[:LIKES]->(lebanese),
  (karin)-[:LIKES]->(italian)

Una vez creados los nodos procedemos a correr la consulta que me permita obtener la semejanza de jaccard entre dos persona dependiendo la comida que les guste en donde partira de obtener el numero total de comidas que tienen en comun en gusto las dos personas  dividido por el numero total de comidas que les gusta cada uno. 
Para utilizar este metodo 

Luego de crear nuestros nodos procedemos a correr la consulta que nos permite determinar la similitud que tienen dos personas 

MATCH (p1:Person {name: 'Karin'})-[:LIKES]->(cuisine1)
WITH p1, collect(id(cuisine1)) AS p1Cuisine
MATCH (p2:Person {name: "Arya"})-[:LIKES]->(cuisine2)
WITH p1, p1Cuisine, p2, collect(id(cuisine2)) AS p2Cuisine
RETURN p1.name AS from,
       p2.name AS to,
       gds.alpha.similarity.jaccard(p1Cuisine, p2Cuisine) AS similarity

In [11]:
from neo4j import GraphDatabase

uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "artificial"),encrypted=False)
    
nodo1 = [] 
nodo2 = [] 
similitud = []
def get_mo(tx):
    result = tx.run("MATCH (p1:Person {name: 'Karin'})-[:LIKES]->(cuisine1) "
                    "WITH p1, collect(id(cuisine1)) AS p1Cuisine "
                    "MATCH (p2:Person {name: 'Arya'})-[:LIKES]->(cuisine2) "
                    "WITH p1, p1Cuisine, p2, collect(id(cuisine2)) AS p2Cuisine "
                    "RETURN p1.name AS from, "
                    "p2.name AS to, "
                    "gds.alpha.similarity.jaccard(p1Cuisine, p2Cuisine) AS similarity ")
    for record in result:
        nodo1.append(record["from"])
        nodo2.append(record["to"]) 
        similitud.append(record["similarity"])

with driver.session() as session:
    ciudades = session.read_transaction(get_mo)
    print("Solución")
    for i in range(len(nodo1)):
        print(nodo1[i], " -> ", nodo2[i]," -> ",similitud[i])                       
driver.close()



Solución
Karin  ->  Arya  ->  0.6666666666666666


## Ejemplificacion usando datos reales.

Para realizar el ejemplo porcedemos a realizar nuestros nodos para ello utilizaremos los lugares de quito que en este caso seran Lugares Turisticos que han gustado a diferentes personas 

A continuacion obtendremos las similitudes entre las diferentes personas, las cuales previamente han especificado los parques que les han gustado.

## Resultados y analisis. 

In [14]:
from neo4j import GraphDatabase

uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "artificial"),encrypted=False)
    
nodo1 = [] 
nodo2 = [] 
similitud = []
def get_mo(tx):
    result = tx.run("MATCH (p:Persona)-[:LIKES]->(Parque) "
                    "WITH {item:id(p), categories: collect(id(Parque))} AS userData "
                    "WITH collect(userData) AS data "
                    "CALL gds.alpha.ml.ann.stream({ "
                    "nodeProjection: '*', "
                    "relationshipProjection: '*', "
                    "data: data, "
                    "algorithm: 'jaccard', "
                    "similarityCutoff: 0.1, "
                    "concurrency: 1 "
                        "}) "
                    "YIELD item1, item2, similarity "
                    "return gds.util.asNode(item1).name AS from, gds.util.asNode(item2).name AS to, similarity "
                    "ORDER BY similarity DESC ")
    
    for record in result:
        nodo1.append(record["from"])
        nodo2.append(record["to"]) 
        similitud.append(record["similarity"])

with driver.session() as session:
    ciudades = session.read_transaction(get_mo)
    print("Solución")
    for i in range(len(nodo1)):
        print(nodo1[i], " -> ", nodo2[i]," -> ",similitud[i])                       
driver.close()

Solución
persona41  ->  persona190  ->  0.5
persona18  ->  persona2  ->  0.3333333333333333
persona50  ->  persona154  ->  0.3333333333333333
persona103  ->  persona149  ->  0.3333333333333333
persona113  ->  persona195  ->  0.3333333333333333
persona142  ->  persona121  ->  0.3333333333333333
persona179  ->  persona20  ->  0.3333333333333333
persona3  ->  persona103  ->  0.2857142857142857
persona134  ->  persona76  ->  0.2857142857142857
persona197  ->  persona185  ->  0.2857142857142857
persona69  ->  persona119  ->  0.25
persona70  ->  persona183  ->  0.25
persona129  ->  persona128  ->  0.25
persona139  ->  persona134  ->  0.25
persona152  ->  persona58  ->  0.25
persona189  ->  persona70  ->  0.25
persona189  ->  persona99  ->  0.25
persona58  ->  persona97  ->  0.23076923076923078
persona33  ->  persona131  ->  0.2222222222222222
persona170  ->  persona59  ->  0.2222222222222222
persona14  ->  persona192  ->  0.2
persona15  ->  persona78  ->  0.2
persona41  ->  persona127  ->  0

## Conclusiones.
Este tipo de algoritmo es implementado para obtener la similitud entre nodos y asi poder sugerir a nodos cercanos diferentes gustos o propiedades de un nodo en particular dependiendo de cuanta similitud se tenga entre nodos. 

### Algoritmo de Propagación de etiquetas

El algoritmo de propagación de etiquetas (LPA) es un algoritmo rápido para encontrar comunidades en un gráfico. Detecta estas comunidades utilizando la estructura de la red por sí sola como su guía, y no requiere una función objetiva predefinida o información previa sobre las comunidades. 
Una característica interesante de LPA es que a los nodos se les pueden asignar etiquetas preliminares para reducir la gama de soluciones generadas. Esto significa que se puede utilizar como forma semi-supervisada de encontrar comunidades donde elegimos a mano algunas comunidades iniciales.

### Descripcion del Algoritmo

LPA funciona propagando etiquetas a través de la red y formando comunidades basadas en este proceso de propagación de etiquetas.

La intuición detrás del algoritmo es que una sola etiqueta puede convertirse rápidamente en dominante en un grupo de nodos densamente conectado, pero tendrá problemas para cruzar una región escasamente conectada. Las etiquetas quedarán atrapadas dentro de un grupo de nodos densamente conectado, y los nodos que terminan con la misma etiqueta cuando los algoritmos terminan se pueden considerar parte de la misma comunidad.

El algoritmo funciona de la siguiente manera:

- Cada nodo se inicializa con una etiqueta de comunidad única (un identificador).
- Estas etiquetas se propagan a través de la red.
- En cada iteración de propagación, cada nodo actualiza su etiqueta a la que pertenece el número máximo de sus vecinos. Los lazos se rompen arbitrariamente pero determinísticamente.
- El LPA alcanza la convergencia cuando cada nodo tiene la escritura de la etiqueta mayoritaria de sus vecinos.
- El LPA se detiene si se logra la convergencia o el número máximo definido por el usuario de las iteraciones.


A medida que las etiquetas se propagan, los grupos de nodos densamente conectados alcanzan rápidamente un consenso sobre una etiqueta única. Al final de la propagación sólo quedarán unas pocas etiquetas - la mayoría habrá desaparecido. Se dice que los nodos que tienen la misma etiqueta de comunidad en la convergencia pertenecen a la misma comunidad. 

¿Cuando se debe utilizar propagacion de etiquetas? 
- La propagación de etiquetas se ha utilizado para asignar polaridad de tweets, como parte del análisis semántico que utiliza etiquetas de semilla de un clasificador entrenado para detectar emoticonos positivos y negativos en combinación con el gráfico de seguidores de Twitter.
- La propagación de etiquetas se ha utilizado para estimar combinaciones potencialmente peligrosas de medicamentos para co-prescribir a un paciente, basado en la similitud química y los perfiles de efectos secundarios. El estudio se encuentra en la predicción de propagación de etiquetas de interacciones fármaco-drogas basadas en efectos secundarios clínicos."
- La propagación de etiquetas se ha utilizado para inferir características de expresiones en un diálogo para un modelo de aprendizaje automático para realizar un seguimiento de la intención del usuario con la ayuda de un gráfico de conocimiento de Wikidata de conceptos y sus relaciones. 

## Ejemplo Sencillo


Para realizar un ejemplo sencillo procedemos a crear los siguientes nodos con sus relaciones en la base de datos  
CREATE (alice:User {name: 'Alice', seed_label: 52})
CREATE (bridget:User {name: 'Bridget', seed_label: 21})
CREATE (charles:User {name: 'Charles', seed_label: 43})
CREATE (doug:User {name: 'Doug', seed_label: 21})
CREATE (mark:User {name: 'Mark', seed_label: 19})
CREATE (michael:User {name: 'Michael', seed_label: 52})

CREATE (alice)-[:FOLLOW {weight: 1}]->(bridget)
CREATE (alice)-[:FOLLOW {weight: 10}]->(charles)
CREATE (mark)-[:FOLLOW {weight: 1}]->(doug)
CREATE (bridget)-[:FOLLOW {weight: 1}]->(michael)
CREATE (doug)-[:FOLLOW {weight: 1}]->(mark)
CREATE (michael)-[:FOLLOW {weight: 1}]->(alice)
CREATE (alice)-[:FOLLOW {weight: 1}]->(michael)
CREATE (bridget)-[:FOLLOW {weight: 1}]->(alice)
CREATE (michael)-[:FOLLOW {weight: 1}]->(bridget)
CREATE (charles)-[:FOLLOW {weight: 1}]->(doug)

Una vez creados los nodos procedemos a correr la consulta que me permita obtener y asignar la comunidad a cada nodo en donde se toma en cuenta los nodos mas cercanos. 

Para utilizar este metodo utilizaremos la siguiente sentencia 

CALL gds.labelPropagation.stream('myGraph')
YIELD nodeId, communityId AS Community
RETURN gds.util.asNode(nodeId).name AS Name, Community
ORDER BY Community, Name


In [19]:
from neo4j import GraphDatabase

uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "artificial"),encrypted=False)
    
nodo1 = [] 
nodo2 = [] 
def get_mo(tx):
    result = tx.run("CALL gds.labelPropagation.stream('myGraphEjemplo') "
                    "YIELD nodeId, communityId AS Community "
                    "RETURN gds.util.asNode(nodeId).name AS Name, Community "
                    "ORDER BY Community, Name ")
    for record in result:
        nodo1.append(record["Name"])
        nodo2.append(record["Community"]) 

with driver.session() as session:
    ciudades = session.read_transaction(get_mo)
    print("Solución")
    for i in range(len(nodo1)):
        print(nodo1[i], " -> ", nodo2[i],)                       
driver.close()



Solución
Alice  ->  11
Bridget  ->  11
Michael  ->  11
Charles  ->  14
Doug  ->  14
Mark  ->  14


Como podemos observar se les ha asignado una comunidad a cada uno de los nodos en donde como se ha mencionado anteriormente se toma en cuenta los nodos que mas cercanos a otros

## Ejemplificacion usando datos reales.

Para realizar el ejemplo porcedemos a realizar nuestros nodos para ello utilizaremos los lugares de quito que en este caso seran Parques que se encuetran almacenados con sus propiedades en la base de datos.

A continuacion obtendremos las comunidades que se les ha asignado a los parques que se ecuentran guardados en la base de datos 

## Resultados y analisis. 

In [21]:
from neo4j import GraphDatabase

uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "artificial"),encrypted=False)
    
nodo1 = [] 
nodo2 = [] 
def get_mo(tx):
    result = tx.run("CALL gds.labelPropagation.stream('myGraph') "
                    "YIELD nodeId, communityId AS Community "
                    "RETURN gds.util.asNode(nodeId).name AS Name, Community "
                    "ORDER BY Community, Name ")
    for record in result:
        nodo1.append(record["Name"])
        nodo2.append(record["Community"]) 

with driver.session() as session:
    ciudades = session.read_transaction(get_mo)
    print("Solución")
    for i in range(len(nodo1)):
        print(nodo1[i], " -> ", nodo2[i],)                       
driver.close()



Solución
Parque Calicanto  ->  27
Parque Liquido  ->  27
Terminal Terrestre Quitumbe  ->  27
Parque 6 de Junio  ->  33
Parque Conocoto  ->  33
Parque Metropolitano Del Sur  ->  33
Parque Metropolitano La Armeria  ->  33
Parque Recreacional La Moya  ->  34
Parque de Los Planetas  ->  34
Parque EL Turismo  ->  35
Parque Jerusalen Alangasi  ->  35
Parque Lineal Selva Alegre  ->  36
Parque Metropolitano Chilibulo  ->  37
Parque Cumanda  ->  46
Parque Guapulo  ->  46
Parque Itichimbia  ->  46
Parque La Alameda  ->  46
Parque Las Cuadras  ->  46
Parque Lineal Machangara  ->  46
Parque Lineal Quito Sur  ->  46
Parque Los Algarrobos  ->  46
Parque Solanda  ->  46
Parque Central Tumbaaco  ->  48
Parque de Puembo  ->  48
Parque Metropolitano Guanguiltagua  ->  49
Parque Japon  ->  52
Parque Nayon  ->  56
Parque Arqueol�gico Rumipamba  ->  62
Parque Bicentenario  ->  62
Parque Carcelen Alto  ->  62
Parque El Ejido  ->  62
Parque La Carolina  ->  62
Parque Los Mastodontes  ->  62
Parque Metropolit

## Conclusiones.
Este tipo de algoritmo es implementado para poder asignar a un serie de nodos que se encuentran a otros cercanos una comunidad que los diferencie de otros nodos que se encuentren mas leganos a los mismo, asi permitiendo que sea mas facil identificar nodos cercanos a otros mediante la comunidad que se le ha asignado a cada uno de los mismos.